# Robot Localization

In this exercise we will apply basic concepts of probability and statistics to help a robot localise itself. Robot localization is an important task for any mobile robot as it needs to know where in the world it is currently situated in.

You may think robot localization is a very easy problem as you can use GPS. But no! GPS has errors in localization upto a few metres which is very inaccurate for self driving cars.

Hence, the robot needs to use sensors to locate itself. And sensors are noisy! Even the motion of a robot is uncertain.

For the purposes of this exercise, let us assume that the world is a one dimensional world with $N$ grid cells. Each grid cell is colored either red or green which can be measured using a sensor located in the robot. However, the sensor measurements are noisy and hence we need to take a probabilitic approach to determine in which of these grid cells the robot is situated in at a given instant of time.

$$
Red \;|\; Green \;|\;  Red \;|\;  Green\;|\; Red
$$

The robot can move either left or right in this **cyclic** world. This motion is uncertain as well prompting us again to take a probabilistic approach. For example you may instruct the robot to move $5$ cells. But it may move $4/6$ cells due to a wheel slip.

When the robot is switched on, it is in a state of maximum confusion. It doesn't know where it is situated. We will model this state of maximum confusion as uniform distribution. 
In other words, the robot has equal belief of being present in any of the cells.

Complete the below function to return an array where the $i^{th}$ array element is the probability that the robot is in the $i^{th}$ grid cell at time instant $0$ when the robot is switched on.

In [0]:
# Modify the empty list, p, so that it becomes a UNIFORM probability
# distribution over n grid cells, as expressed in a list of 
# n probabilities.
def init_distribution(n):
    p = []
    p = [1/n for i in range(n)]
    return p

In [5]:
init_distribution(5)

[0.2, 0.2, 0.2, 0.2, 0.2]

The node is allowed to sense its environment using sensors. These measurements are noisy. 

In a red cell,
$$
P(red | red) = 0.75
$$
Hence,
$$
P(green | red) = 0.25
$$

In a green cell,
$$
P(red | green) = 0.25
$$
Hence,
$$
P(green | green) = 0.75
$$

In [28]:
# Modify the code below so that the function sense, which 
# takes p and Z as inputs, will output the normalized 
# probability distribution, q, after multiplying the entries 
# in p according to the color in the corresponding cell in world,
# followed by normalization of the probability values
import numpy as np
world=['green', 'red', 'red', 'green', 'green']
p = [0.2, 0.2, 0.2, 0.2, 0.2]
same = 0.75
diff = 0.25
def sense(p, Z):
    q = []
    for i in range(len(p)):
    #   if i == 0: 
    #     previous = world[len(p) - 1]
    #   else:
    #     previous = world[i - 1]

      previous = world[i]
      if(Z == previous):
        q.append(same * p[i])
      else:
        q.append(diff * p[i])
    q = q/np.sum(q)
    return q
            
print(sense(p, 'red'))
print(sense(p, 'green'))

[0.11111111 0.33333333 0.33333333 0.11111111 0.11111111]
[0.27272727 0.09090909 0.09090909 0.27272727 0.27272727]


Update probability for multiple measurements.

In [30]:
# Modify the code so that it updates the probability twice
# and gives the posterior distribution after both 
# measurements are incorporated. Make sure that your code 
# allows for any sequence of measurement of any length.

# Do not modify the sense function. Call it correctly.

p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']

for i in measurements:
  p = sense(p,i) 
print(p)

[0.2 0.2 0.2 0.2 0.2]


In [31]:
# Program a function that returns a new distribution 
# q, shifted to the right by U units. If U = 0, q should 
# be the same as p.

def move(p, U):
    q = []
    U = U % len(p)
    q = np.roll(p, U)
    return q

move([0.11, 0.33, 0.33, 0.11, 0.11], 1)

array([0.11, 0.11, 0.33, 0.33, 0.11])

**Solution:**

We can apply the Theorem of total probability at each grid cell given the robot instruction. For example, we are interested in computing the probability that the robot is at grid cell $2$. Now the robot can come here from the following gird cells:
- Grid cell $1$: The instruction is to move right by $2$, and hence a robot at cell $1$ can reach this grid cell with a probability of $0.1$(this is by undershooting)
- Grid cell $0$ : The instruction is to move right by $2$, and hence a robot at cell $0$ can reach this grid cell with a probability of $0.8$(this is case when robot motion is exact)
- Grid cell $4$ : The instruction is to move right by $2$, and hence a robot at cell $4$ can reach this grid cell with a probability of $0.1$(this is by overshooting). Remember that the world is cyclic.

Required Probability ($P$) is given as follows:
\begin{equation}
P = 1.0 * 0.1 + 0.0 * 0.8 + 0.0 * 0.1 = 0.1
\end{equation}

In [34]:
# Modify the move function to accommodate the added 
# probabilities of overshooting or undershooting 
# the intended destination.

def move(p, U):
    q = np.array([])
    n = len(p)
    for i in range(n):
        q = np.append(q, p[(i-U)%n]*0.8 + p[(i-U-1)%n]*0.1 + p[(i-U+1)%n]*0.1)
    return q

move([0, 1.0, 0, 0, 0], 1)

array([0. , 0.1, 0.8, 0.1, 0. ])

In [36]:
# Write code that makes the robot move twice and then prints 
# out the resulting distribution, starting with the initial 
# distribution p = [0, 1, 0, 0, 0]

# Call the function appropriately from here
move(move([0, 1.0, 0, 0, 0], 1), 1)

array([0.01, 0.01, 0.16, 0.66, 0.16])

In [38]:
# Write code that moves 1000 times and then prints the 
# resulting probability distribution.

p = [0, 1, 0, 0, 0]
for i in range(1000):
    p = move(p, 1)
print(p)

[0.2 0.2 0.2 0.2 0.2]


In [41]:
#Given the list motions=[1,1] which means the robot 
#moves right and then right again, compute the posterior 
#distribution if the robot first senses red, then moves 
#right one, then senses green, then moves right again, 
#starting with a uniform prior distribution.

world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
motions = [1, 1]

p = init_distribution(len(world))
for i in range(len(motions)):
    q = sense(p, measurements[i])
    qNew = move(q, motions[i])
    p = qNew
print(p)


[0.21157895 0.15157895 0.08105263 0.16842105 0.38736842]


### Two Dimensional Robot Localization

In [0]:
# Finsh this